In [ ]:
from argparse import Namespace
from classifier import *

from dataset import *
from vectorizer import *

import torch

In [ ]:
def use_cuda():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

args = Namespace (
    in_features=None,
    out_units=1,
    dataset=None,
    optimizer="Adam",
    criterion="bce_logits",
    batch_size=128,
    learning_rate=0.001,
    num_epochs=1,
    device=use_cuda(),
    embed_dim=300,
    freeze_embedding=False,
    filter_sizes=[3, 4, 5],
    num_filters=[100, 100, 100],
    pretrained_embedding=None, 
    hidden_size=2, # number of features in hidden state
    num_layers=1 # number of stacked lstm layers
)

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/tweets_products.csv")
args.in_features = args.dataset.vectorizer.max_padding

### BOW

In [ ]:
bow = BOWClassifier(args=args)
bow.setup()
bow.fit()

In [ ]:
loss, acc, f1 = bow.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/tweets_products.csv")
loss, acc, f1 = bow.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/IMDB-dataset.csv", vectorizer_mode="padded")
args.in_features = len(args.dataset.vectorizer.text_vocab)
args.pretrained_embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")

### CNN

In [ ]:
cnn_classifier = CNNClassifier(args=args)
cnn_classifier.setup()
cnn_classifier.fit()

In [ ]:
loss, acc, f1 = cnn_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/tweets_products.csv", vectorizer_mode="padded")
embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")
embedding = embedding.to(args.device)
cnn_classifier.embedding = nn.Embedding.from_pretrained (
    embedding,
    freeze=args.freeze_embedding
)
loss, acc, f1 = cnn_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
# torch.save(cnn_classifier.state_dict(), "./cnn_classifier.pth")

### LSTM

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/IMDB-dataset.csv", vectorizer_mode="padded")
args.in_features = len(args.dataset.vectorizer.text_vocab)
args.pretrained_embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")

In [ ]:
lstm_classifier = LSTMClassifier(args)
lstm_classifier.setup()
lstm_classifier.fit()

In [ ]:
loss, acc, f1 = lstm_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/tweets_products.csv", vectorizer_mode="padded")
embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")
embedding = embedding.to(args.device)
lstm_classifier.embedding = nn.Embedding.from_pretrained (
    args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec"),
    freeze=args.freeze_embedding
)


In [ ]:
loss, acc, f1 = lstm_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
# torch.save(lstm_classifier.state_dict(), "./lstm_classifier.pth")